In [1]:
import stanza
import pandas as pd
import ast
from collections import Counter

In [2]:
form = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5",
                   key="reduced_music_form")
sentence = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\new_unigram_data.h5",
                   key="sentence")

In [3]:
ids = form.loc[form["Desired"] == 1, "Review id"].unique()
form = form.loc[form["Review id"].isin(ids)]

In [4]:
sentence.loc[:, "Text"] = sentence.loc[:, "Text"].apply(ast.literal_eval)
sentence = sentence.explode("Text")
sentence["Sentence id"] = sentence.groupby(["Review id"]).cumcount()

In [5]:
sentence = sentence.loc[sentence["Review id"].isin(ids)].explode("Text")

In [6]:
sentence["Word id"] = sentence.groupby("Review id").cumcount()

In [7]:
form = pd.merge(form, sentence, on=["Artist", "Album", "Platform", "Review id", "Word id"], how="inner").drop(columns=["Word"])

In [8]:
form = form.groupby(["Review id", "Sentence id"]).agg({"Text": " ".join, "Desired": list}).reset_index()

In [9]:
form["Desired"] = form.loc[:, "Desired"].apply(lambda x: [desired == 1 for desired in x])

In [34]:
nlp_d = stanza.Pipeline(lang='en', processors='pos,depparse,lemma,tokenize,mwt', tokenize_pretokenized=True)

2023-02-15 15:36:05 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-02-15 15:36:05 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-02-15 15:36:06 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2023-02-15 15:36:06 INFO: Use device: cpu
2023-02-15 15:36:06 INFO: Loading: tokenize
2023-02-15 15:36:06 INFO: Loading: pos
2023-02-15 15:36:07 INFO: Loading: lemma
2023-02-15 15:36:07 INFO: Loading: depparse
2023-02-15 15:36:07 INFO: Done loading processors!


In [11]:
# Counts all dependencies in and out of words that have a dependency with desired and undesired adjectives
desired_deps1 = []
undesired_deps1 = []

for index, row in form.iterrows():
    deps = nlp_d_d(row.loc["Text"]).sentences[0].dependencies
    desireds = [i + 1 for i, x in enumerate(row.loc["Desired"]) if x]

    for dep in deps:
        if dep[2].id in desireds and dep[2].xpos == "JJ":
            dep_id = dep[2].id
        
            for dep_sub in deps:
                if dep_sub[0].id == dep_id:
                    desired_deps1.append(dep_sub[2].deprel)
                if dep_sub[2].id == dep_id:
                    desired_deps1.append(dep_sub[0].deprel)
                    
        elif dep[2].xpos == "JJ":
            dep_id = dep[2].id
                
            for dep_sub in deps:
                if dep_sub[0].id == dep_id:
                    undesired_deps1.append(dep_sub[2].deprel)
                if dep_sub[2].id == dep_id:
                    undesired_deps1.append(dep_sub[0].deprel)
                    
    

In [12]:
u_dic1 = sorted({k: v / len(undesired_deps1) for k, v in Counter(undesired_deps1).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
d_dic1 = sorted({k: v / len(desired_deps1) for k, v in Counter(desired_deps1).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)

In [13]:
u_dic1

[('obl', 0.16639741518578352),
 ('nmod', 0.11447034387260559),
 ('obj', 0.11031617816755135),
 ('nsubj', 0.10846988229863835),
 ('advmod', 0.09416108931456266),
 ('conj', 0.06669743826448188),
 ('root', 0.04892684052619432),
 ('cop', 0.035079621509346876),
 ('advcl', 0.03115624278790676),
 ('cc', 0.030463881837064388),
 ('xcomp', 0.0210016155088853),
 ('mark', 0.015462727902146318),
 ('amod', 0.013616432033233325),
 ('det', 0.012924071082390954),
 ('compound', 0.01269328409877683),
 ('case', 0.01269328409877683),
 ('acl:relcl', 0.011539349180706208),
 ('ccomp', 0.011077775213477959),
 ('parataxis', 0.010846988229863836),
 ('nsubj:pass', 0.0073851834756519736),
 ('aux', 0.0073851834756519736),
 ('appos', 0.006231248557581353),
 ('acl', 0.006231248557581353),
 ('obl:npmod', 0.0043849526886683594),
 ('nmod:poss', 0.004154165705054235),
 ('obl:tmod', 0.0036925917378259868),
 ('csubj', 0.0032310177705977385),
 ('fixed', 0.003000230786983614),
 ('punct', 0.0020770828525271175),
 ('expl', 0.0

In [14]:
d_dic1

[('obl', 0.17710843373493976),
 ('conj', 0.12289156626506025),
 ('advmod', 0.10843373493975904),
 ('nsubj', 0.10240963855421686),
 ('obj', 0.0927710843373494),
 ('nmod', 0.0891566265060241),
 ('root', 0.060240963855421686),
 ('cc', 0.04939759036144578),
 ('cop', 0.03614457831325301),
 ('xcomp', 0.01927710843373494),
 ('amod', 0.018072289156626505),
 ('advcl', 0.016867469879518072),
 ('compound', 0.00963855421686747),
 ('acl:relcl', 0.00963855421686747),
 ('mark', 0.008433734939759036),
 ('nsubj:pass', 0.007228915662650603),
 ('appos', 0.007228915662650603),
 ('case', 0.007228915662650603),
 ('parataxis', 0.006024096385542169),
 ('obl:npmod', 0.006024096385542169),
 ('acl', 0.004819277108433735),
 ('obl:agent', 0.0036144578313253013),
 ('det', 0.0036144578313253013),
 ('ccomp', 0.0036144578313253013),
 ('nmod:poss', 0.0036144578313253013),
 ('csubj', 0.0024096385542168677),
 ('aux', 0.0024096385542168677),
 ('discourse', 0.0012048192771084338),
 ('cc:preconj', 0.0012048192771084338),
 (

In [15]:
# Counts all direct dependencies with desired and undesired adjectives
desired_deps2 = []
undesired_deps2 = []

for index, row in form.iterrows():
    deps = nlp_d(row.loc["Text"]).sentences[0].dependencies
    desireds = [i + 1 for i, x in enumerate(row.loc["Desired"]) if x]

    for dep in deps:
        if dep[0].id in desireds and dep[0].xpos == "JJ":
            desired_deps2.append(dep[2].deprel)
                    
        elif dep[0].xpos == "JJ":
            undesired_deps2.append(dep[2].deprel)

In [16]:
u_dic2 = sorted({k: v / len(undesired_deps2) for k, v in Counter(undesired_deps2).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
d_dic2 = sorted({k: v / len(desired_deps2) for k, v in Counter(desired_deps2).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)

In [17]:
u_dic2

[('advmod', 0.25440806045340053),
 ('nsubj', 0.09697732997481108),
 ('cop', 0.09571788413098237),
 ('obl', 0.09445843828715365),
 ('cc', 0.08312342569269521),
 ('conj', 0.07304785894206549),
 ('advcl', 0.05478589420654912),
 ('mark', 0.04219143576826197),
 ('det', 0.03526448362720403),
 ('case', 0.03400503778337532),
 ('aux', 0.020151133501259445),
 ('xcomp', 0.015113350125944584),
 ('parataxis', 0.014483627204030227),
 ('nmod:poss', 0.010705289672544081),
 ('ccomp', 0.009445843828715366),
 ('nmod', 0.009445843828715366),
 ('csubj', 0.00818639798488665),
 ('fixed', 0.00818639798488665),
 ('obl:npmod', 0.0069269521410579345),
 ('punct', 0.005667506297229219),
 ('amod', 0.005667506297229219),
 ('compound', 0.005667506297229219),
 ('expl', 0.004408060453400504),
 ('acl', 0.0025188916876574307),
 ('obj', 0.0025188916876574307),
 ('discourse', 0.001889168765743073),
 ('acl:relcl', 0.001889168765743073),
 ('obl:tmod', 0.0012594458438287153),
 ('cc:preconj', 0.0012594458438287153),
 ('nummod'

In [19]:
desired_outgoing3 = []
desired_incoming3 = []
undesired_outgoing3 = []
undesired_incoming3 = []

for index, row in form.iterrows():
    deps = nlp_d(row.loc["Text"]).sentences[0].dependencies
    desireds = [i + 1 for i, x in enumerate(row.loc["Desired"]) if x]

    for dep in deps:
        if dep[2].id in desireds and dep[2].xpos == "JJ":
            dep_id = dep[2].id
        
            for dep_sub in deps:
                if dep_sub[0].id == dep_id:
                    desired_incoming3.append(dep_sub[2].deprel)
                if dep_sub[2].id == dep_id:
                    desired_outgoing3.append(dep_sub[0].deprel)
                    
        elif dep[2].xpos == "JJ":
            dep_id = dep[2].id
                
            for dep_sub in deps:
                if dep_sub[0].id == dep_id:
                    undesired_incoming3.append(dep_sub[2].deprel)
                if dep_sub[2].id == dep_id:
                    undesired_outgoing3.append(dep_sub[0].deprel)

In [20]:
uo_dic3 = sorted({k: v / len(undesired_outgoing3) for k, v in Counter(undesired_outgoing3).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
ui_dic3 = sorted({k: v / len(undesired_incoming3) for k, v in Counter(undesired_incoming3).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
do_dic3 = sorted({k: v / len(desired_outgoing3) for k, v in Counter(desired_outgoing3).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
di_dic3 = sorted({k: v / len(desired_incoming3) for k, v in Counter(desired_incoming3).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)

In [21]:
ui_dic3

[('advmod', 0.25440806045340053),
 ('nsubj', 0.09697732997481108),
 ('cop', 0.09571788413098237),
 ('obl', 0.09445843828715365),
 ('cc', 0.08312342569269521),
 ('conj', 0.07304785894206549),
 ('advcl', 0.05478589420654912),
 ('mark', 0.04219143576826197),
 ('det', 0.03526448362720403),
 ('case', 0.03400503778337532),
 ('aux', 0.020151133501259445),
 ('xcomp', 0.015113350125944584),
 ('parataxis', 0.014483627204030227),
 ('nmod:poss', 0.010705289672544081),
 ('ccomp', 0.009445843828715366),
 ('nmod', 0.009445843828715366),
 ('csubj', 0.00818639798488665),
 ('fixed', 0.00818639798488665),
 ('obl:npmod', 0.0069269521410579345),
 ('punct', 0.005667506297229219),
 ('amod', 0.005667506297229219),
 ('compound', 0.005667506297229219),
 ('expl', 0.004408060453400504),
 ('acl', 0.0025188916876574307),
 ('obj', 0.0025188916876574307),
 ('discourse', 0.001889168765743073),
 ('acl:relcl', 0.001889168765743073),
 ('obl:tmod', 0.0012594458438287153),
 ('cc:preconj', 0.0012594458438287153),
 ('nummod'

In [22]:
di_dic3

[('advmod', 0.3010033444816054),
 ('conj', 0.13712374581939799),
 ('cc', 0.13712374581939799),
 ('cop', 0.10033444816053512),
 ('nsubj', 0.09364548494983277),
 ('obl', 0.06688963210702341),
 ('advcl', 0.033444816053511704),
 ('mark', 0.023411371237458192),
 ('case', 0.020066889632107024),
 ('obl:npmod', 0.016722408026755852),
 ('det', 0.010033444816053512),
 ('nmod:poss', 0.010033444816053512),
 ('csubj', 0.006688963210702341),
 ('aux', 0.006688963210702341),
 ('obj', 0.006688963210702341),
 ('xcomp', 0.006688963210702341),
 ('acl:relcl', 0.006688963210702341),
 ('amod', 0.0033444816053511705),
 ('ccomp', 0.0033444816053511705),
 ('discourse', 0.0033444816053511705),
 ('cc:preconj', 0.0033444816053511705),
 ('obl:tmod', 0.0033444816053511705)]

In [23]:
uo_dic3

[('obl', 0.20801457194899817),
 ('nmod', 0.17522768670309655),
 ('obj', 0.1726775956284153),
 ('nsubj', 0.1151183970856102),
 ('root', 0.07723132969034609),
 ('conj', 0.06302367941712204),
 ('xcomp', 0.024408014571948997),
 ('amod', 0.018214936247723135),
 ('advcl', 0.017486338797814208),
 ('acl:relcl', 0.017122040072859744),
 ('compound', 0.016757741347905284),
 ('ccomp', 0.012021857923497269),
 ('nsubj:pass', 0.011657559198542805),
 ('appos', 0.009836065573770493),
 ('parataxis', 0.008743169398907104),
 ('acl', 0.008378870673952642),
 ('obl:tmod', 0.005100182149362477),
 ('obl:npmod', 0.0029143897996357013),
 ('obl:agent', 0.0014571948998178506),
 ('advmod', 0.0014571948998178506),
 ('nmod:poss', 0.00036429872495446266),
 ('case', 0.00036429872495446266),
 ('nummod', 0.00036429872495446266),
 ('csubj', 0.00036429872495446266),
 ('iobj', 0.00036429872495446266)]

In [24]:
do_dic3

[('obl', 0.2391713747645951),
 ('obj', 0.14124293785310735),
 ('nmod', 0.1393596986817326),
 ('conj', 0.11487758945386065),
 ('nsubj', 0.10734463276836158),
 ('root', 0.09416195856873823),
 ('xcomp', 0.026365348399246705),
 ('amod', 0.026365348399246705),
 ('compound', 0.015065913370998116),
 ('nsubj:pass', 0.011299435028248588),
 ('acl:relcl', 0.011299435028248588),
 ('appos', 0.011299435028248588),
 ('parataxis', 0.009416195856873822),
 ('acl', 0.007532956685499058),
 ('advcl', 0.007532956685499058),
 ('obl:agent', 0.005649717514124294),
 ('ccomp', 0.003766478342749529)]

In [25]:
desired_outgoing4 = []
desired_incoming4 = []
undesired_outgoing4 = []
undesired_incoming4 = []

for index, row in form.iterrows():
    deps = nlp_d(row.loc["Text"]).sentences[0].dependencies
    desireds = [i + 1 for i, x in enumerate(row.loc["Desired"]) if x]

    for dep in deps:
        if dep[2].id in desireds:
            if dep[2].xpos == "JJ":
                desired_incoming4.append(dep[1])
        elif dep[2].xpos == "JJ":
            undesired_incoming4.append(dep[1])
        elif dep[0].id in desireds:
            if dep[0].xpos == "JJ":
                desired_outgoing4.append(dep[1])
        elif dep[0].xpos == "JJ":
            undesired_outgoing4.append(dep[1])

In [26]:
uo_dic4 = sorted({k: v / len(undesired_outgoing4) for k, v in Counter(undesired_outgoing4).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
ui_dic4 = sorted({k: v / len(undesired_incoming4) for k, v in Counter(undesired_incoming4).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
do_dic4 = sorted({k: v / len(desired_outgoing4) for k, v in Counter(desired_outgoing4).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
di_dic4 = sorted({k: v / len(desired_incoming4) for k, v in Counter(desired_incoming4).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)

In [27]:
uo_dic4

[('advmod', 0.272108843537415),
 ('nsubj', 0.10340136054421768),
 ('cop', 0.10340136054421768),
 ('obl', 0.09931972789115646),
 ('cc', 0.08979591836734693),
 ('advcl', 0.05442176870748299),
 ('mark', 0.04557823129251701),
 ('det', 0.03741496598639456),
 ('case', 0.036734693877551024),
 ('aux', 0.021768707482993196),
 ('conj', 0.02108843537414966),
 ('xcomp', 0.0163265306122449),
 ('parataxis', 0.014965986394557823),
 ('nmod:poss', 0.011564625850340135),
 ('nmod', 0.01020408163265306),
 ('ccomp', 0.009523809523809525),
 ('csubj', 0.008843537414965987),
 ('fixed', 0.008843537414965987),
 ('punct', 0.006122448979591836),
 ('compound', 0.006122448979591836),
 ('obl:npmod', 0.005442176870748299),
 ('expl', 0.004761904761904762),
 ('acl', 0.0027210884353741495),
 ('obj', 0.0027210884353741495),
 ('discourse', 0.0020408163265306124),
 ('obl:tmod', 0.0013605442176870747),
 ('cc:preconj', 0.0013605442176870747),
 ('acl:relcl', 0.0013605442176870747),
 ('nummod', 0.0006802721088435374)]

In [28]:
do_dic4

[('advmod', 0.32644628099173556),
 ('cc', 0.16942148760330578),
 ('cop', 0.12396694214876033),
 ('nsubj', 0.11570247933884298),
 ('obl', 0.07851239669421488),
 ('advcl', 0.028925619834710745),
 ('mark', 0.028925619834710745),
 ('conj', 0.028925619834710745),
 ('case', 0.02066115702479339),
 ('det', 0.012396694214876033),
 ('nmod:poss', 0.012396694214876033),
 ('csubj', 0.008264462809917356),
 ('aux', 0.008264462809917356),
 ('obj', 0.008264462809917356),
 ('acl:relcl', 0.008264462809917356),
 ('obl:npmod', 0.004132231404958678),
 ('ccomp', 0.004132231404958678),
 ('discourse', 0.004132231404958678),
 ('cc:preconj', 0.004132231404958678),
 ('obl:tmod', 0.004132231404958678)]

In [29]:
ui_dic4

[('amod', 0.8072859744990892),
 ('conj', 0.048816029143897995),
 ('xcomp', 0.036794171220400726),
 ('root', 0.03132969034608379),
 ('advcl', 0.01238615664845173),
 ('obl', 0.010200364298724954),
 ('obj', 0.008378870673952642),
 ('nsubj', 0.0069216757741347905),
 ('nmod', 0.006193078324225865),
 ('acl:relcl', 0.0058287795992714025),
 ('advmod', 0.00546448087431694),
 ('case', 0.004735883424408015),
 ('parataxis', 0.004371584699453552),
 ('ccomp', 0.004007285974499089),
 ('obl:npmod', 0.0036429872495446266),
 ('acl', 0.0014571948998178506),
 ('nsubj:pass', 0.0007285974499089253),
 ('obl:tmod', 0.00036429872495446266),
 ('compound', 0.00036429872495446266),
 ('nmod:poss', 0.00036429872495446266),
 ('mark', 0.00036429872495446266)]

In [30]:
di_dic4

[('amod', 0.807909604519774),
 ('conj', 0.07344632768361582),
 ('xcomp', 0.047080979284369114),
 ('root', 0.02824858757062147),
 ('advcl', 0.01694915254237288),
 ('obl', 0.005649717514124294),
 ('parataxis', 0.005649717514124294),
 ('acl:relcl', 0.003766478342749529),
 ('nmod', 0.003766478342749529),
 ('obj', 0.003766478342749529),
 ('acl', 0.0018832391713747645),
 ('advmod', 0.0018832391713747645)]

In [31]:
desired_outgoing5 = []
desired_incoming5 = []
undesired_outgoing5 = []
undesired_incoming5 = []

for index, row in form.iterrows():
    deps = nlp_d(row.loc["Text"]).sentences[0].dependencies
    desireds = [i + 1 for i, x in enumerate(row.loc["Desired"]) if x]

    for dep in deps:
        
        if dep[2].id in desireds and dep[2].xpos == "JJ":
            dep_id = dep[2].id
            word_deps_incoming = []
            word_deps_outgoing = []
        
            for dep_sub in deps:
                if dep_sub[0].id == dep_id:
                    word_deps_incoming.append(dep_sub[2].deprel)
                if dep_sub[2].id == dep_id:
                    word_deps_outgoing.append(dep_sub[0].deprel)
            desired_incoming5.append(tuple(sorted(tuple(word_deps_incoming))))
            desired_outgoing5.append(tuple(sorted(tuple(word_deps_outgoing))))
                    
        elif dep[2].xpos == "JJ":
            dep_id = dep[2].id
            word_deps_incoming = []
            word_deps_outgoing = []
                
            for dep_sub in deps:
                if dep_sub[0].id == dep_id:
                    word_deps_incoming.append(dep_sub[2].deprel)
                if dep_sub[2].id == dep_id:
                    word_deps_outgoing.append(dep_sub[0].deprel)
            undesired_incoming5.append(tuple(sorted(tuple(word_deps_incoming))))
            undesired_outgoing5.append(tuple(sorted(tuple(word_deps_outgoing))))

In [32]:
uo_dic5 = sorted({k: v / len(undesired_outgoing5) for k, v in Counter(undesired_outgoing5).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
ui_dic5 = sorted({k: v / len(undesired_incoming5) for k, v in Counter(undesired_incoming5).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
do_dic5 = sorted({k: v / len(desired_outgoing5) for k, v in Counter(desired_outgoing5).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
di_dic5 = sorted({k: v / len(desired_incoming5) for k, v in Counter(desired_incoming5).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)

In [33]:
uo_dic5

[(('obl',), 0.20801457194899817),
 (('nmod',), 0.17522768670309655),
 (('obj',), 0.1726775956284153),
 (('nsubj',), 0.1151183970856102),
 (('root',), 0.07723132969034609),
 (('conj',), 0.06302367941712204),
 ((None,), 0.03132969034608379),
 (('xcomp',), 0.024408014571948997),
 (('amod',), 0.018214936247723135),
 (('advcl',), 0.017486338797814208),
 (('acl:relcl',), 0.017122040072859744),
 (('compound',), 0.016757741347905284),
 (('ccomp',), 0.012021857923497269),
 (('nsubj:pass',), 0.011657559198542805),
 (('appos',), 0.009836065573770493),
 (('parataxis',), 0.008743169398907104),
 (('acl',), 0.008378870673952642),
 (('obl:tmod',), 0.005100182149362477),
 (('obl:npmod',), 0.0029143897996357013),
 (('obl:agent',), 0.0014571948998178506),
 (('advmod',), 0.0014571948998178506),
 (('nmod:poss',), 0.00036429872495446266),
 (('case',), 0.00036429872495446266),
 (('nummod',), 0.00036429872495446266),
 (('csubj',), 0.00036429872495446266),
 (('iobj',), 0.00036429872495446266)]

In [34]:
do_dic5

[(('obl',), 0.2391713747645951),
 (('obj',), 0.14124293785310735),
 (('nmod',), 0.1393596986817326),
 (('conj',), 0.11487758945386065),
 (('nsubj',), 0.10734463276836158),
 (('root',), 0.09416195856873823),
 ((None,), 0.02824858757062147),
 (('xcomp',), 0.026365348399246705),
 (('amod',), 0.026365348399246705),
 (('compound',), 0.015065913370998116),
 (('nsubj:pass',), 0.011299435028248588),
 (('acl:relcl',), 0.011299435028248588),
 (('appos',), 0.011299435028248588),
 (('parataxis',), 0.009416195856873822),
 (('acl',), 0.007532956685499058),
 (('advcl',), 0.007532956685499058),
 (('obl:agent',), 0.005649717514124294),
 (('ccomp',), 0.003766478342749529)]

In [35]:
ui_dic5

[((), 0.7373406193078325),
 (('advmod',), 0.05573770491803279),
 (('cc',), 0.02477231329690346),
 (('conj',), 0.012750455373406194),
 (('obl',), 0.00947176684881603),
 (('det',), 0.009107468123861567),
 (('advmod', 'obl'), 0.006557377049180328),
 (('advmod', 'conj'), 0.005100182149362477),
 (('fixed',), 0.004735883424408015),
 (('case', 'det'), 0.004007285974499089),
 (('advcl', 'advmod'), 0.003278688524590164),
 (('mark',), 0.0029143897996357013),
 (('case',), 0.0025500910746812386),
 (('advmod', 'advmod'), 0.0025500910746812386),
 (('advcl',), 0.002185792349726776),
 (('cc', 'obl'), 0.002185792349726776),
 (('advmod', 'cc'), 0.002185792349726776),
 (('advmod', 'cop', 'nsubj', 'obl'), 0.0018214936247723133),
 (('xcomp',), 0.0018214936247723133),
 (('nmod',), 0.0018214936247723133),
 (('cop', 'nsubj', 'obl'), 0.0014571948998178506),
 (('advcl', 'cop', 'nsubj'), 0.0014571948998178506),
 (('cop', 'nsubj'), 0.0014571948998178506),
 (('case', 'nmod'), 0.0014571948998178506),
 (('advcl', 'a

In [36]:
di_dic5

[((), 0.7212806026365348),
 (('advmod',), 0.07156308851224105),
 (('cc',), 0.054613935969868174),
 (('conj',), 0.02071563088512241),
 (('advmod', 'conj'), 0.011299435028248588),
 (('advmod', 'obl'), 0.011299435028248588),
 (('advmod', 'cc'), 0.009416195856873822),
 (('obl:npmod',), 0.007532956685499058),
 (('advcl', 'advmod'), 0.003766478342749529),
 (('advmod', 'conj', 'obl'), 0.003766478342749529),
 (('cop', 'mark', 'nsubj'), 0.003766478342749529),
 (('advmod', 'conj', 'cop', 'nsubj', 'obj'), 0.003766478342749529),
 (('advmod', 'cc', 'cop', 'nsubj'), 0.003766478342749529),
 (('nmod:poss',), 0.003766478342749529),
 (('advcl', 'amod', 'cop', 'nsubj'), 0.0018832391713747645),
 (('advcl', 'advmod', 'conj', 'cop', 'nsubj'), 0.0018832391713747645),
 (('advcl', 'aux', 'conj', 'cop', 'csubj', 'nsubj'), 0.0018832391713747645),
 (('advmod', 'conj', 'cop', 'nsubj'), 0.0018832391713747645),
 (('xcomp',), 0.0018832391713747645),
 (('obl',), 0.0018832391713747645),
 (('advcl', 'cop', 'nsubj'), 0.0

In [37]:
desired6 = []
undesired6 = []

for index, row in form.iterrows():
    deps = nlp_d(row.loc["Text"]).sentences[0].dependencies
    desireds = [i + 1 for i, x in enumerate(row.loc["Desired"]) if x]

    for dep in deps:
        
        if dep[2].id in desireds and dep[2].xpos == "JJ":
            desired6.append(dep[0].xpos)          
        elif dep[2].xpos == "JJ":
            undesired6.append(dep[0].xpos)

In [38]:
d_dic6 = sorted({k: v / len(desired6) for k, v in Counter(desired6).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)
u_dic6 = sorted({k: v / len(undesired6) for k, v in Counter(undesired6).items() if k is not None}.items(), key=lambda x: x[1], reverse=True)

In [39]:
d_dic6

[('NN', 0.5725047080979284),
 ('NNS', 0.2278719397363465),
 ('JJ', 0.0696798493408663),
 ('VBZ', 0.04143126177024482),
 ('NNP', 0.015065913370998116),
 ('VBG', 0.011299435028248588),
 ('VB', 0.011299435028248588),
 ('VBN', 0.009416195856873822),
 ('VBP', 0.007532956685499058),
 ('NNPS', 0.003766478342749529),
 ('PRP', 0.0018832391713747645)]

In [40]:
u_dic6

[('NN', 0.556648451730419),
 ('NNS', 0.23934426229508196),
 ('JJ', 0.03970856102003643),
 ('VBZ', 0.029508196721311476),
 ('NNP', 0.0273224043715847),
 ('VB', 0.021493624772313296),
 ('VBN', 0.013843351548269581),
 ('VBD', 0.010564663023679418),
 ('VBG', 0.009836065573770493),
 ('VBP', 0.008014571948998178),
 ('NNPS', 0.004007285974499089),
 ('CD', 0.0025500910746812386),
 ('JJR', 0.0014571948998178506),
 ('PRP', 0.001092896174863388),
 ('RB', 0.001092896174863388),
 ('IN', 0.0007285974499089253),
 ('RBR', 0.0007285974499089253),
 ('JJS', 0.00036429872495446266),
 ('WP', 0.00036429872495446266)]

In [10]:
test_form = form.loc[form["Review id"] == form["Review id"].min()]

In [53]:
import sys, os

sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())))
from nlp import keywords

def get_instrument_labels(sentence, desired, nlp_d):
    
    instrument_words = keywords.get_instrument_words()
    desireds = [i + 1 for i, x in enumerate(desired) if x]
    sentence = nlp_d(sentence)
    deps = sentence.sentences[0].dependencies
    labels = [0 for _ in range(len(sentence.sentences[0].words))]
    
    for dep in deps:
        if dep[1] == "amod" and dep[2].id in desireds:
            adj_id = dep[2].id
            subj_id = dep[0].id
            subject_text = dep[0].text
            
            for dep in deps:
                if dep[1] == "compound" and dep[0].id == subj_id:
                    subject_text = dep[2].text + " " + subject_text
                                
            if any([x in subject_text for x in instrument_words]):
                labels[adj_id - 1] = 1
                print(labels, [word.text for word in sentence.sentences[0].words])
                
    return labels
            

In [54]:
test_form.loc[:, ["Desired", "Text"]].apply(lambda x: get_instrument_labels(x.loc["Text"], x.loc["Desired"], nlp_d), axis=1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] ["It's", 'the', 'cornerstone', 'of', 'Sleaford', "Mods'", 'sound', 'underscored', 'by', 'double-tracking', 'on', 'crucial', 'phrases', 'the', 'music', 'behind', 'it', 'is', 'loops', 'made', 'by', 'Andrew', 'Fearn', 'the', 'other', 'half', 'of', 'the', 'band', 'mostly', 'from', 'blunt', 'basement-rock', 'bass', 'and', 'drums']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] ['The', 'creepily', 'catchy', 'Tarantula', 'Deadly', 'Cargo', 'which', 'may', 'or', 'may', 'not', 'be', 'a', 'scatological', 'joke', 'would', 'fit', 'right', 'in', 'on', 'the', "Fall's", 'Dragnet', 'especially', "Fearn's", 'asthmatic', 'guitar', 'plinks']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] ['But', "they've", 'adopted', 'its', 'raw', 'elements—crudity', 'spittle', 'black', 'humor', 'and', 'unpretty', 'voices—as', 'durable', 'tools', 'to', 'express', 'discont

0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
12    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
13    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
14    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
15                                               [0, 0]
16                                         [0, 0, 0, 0]
17                                              

[({
    "id": 9,
    "text": "takes",
    "lemma": "take",
    "upos": "VERB",
    "xpos": "VBZ",
    "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
    "head": 0,
    "deprel": "root",
    "misc": "",
    "start_char": 44,
    "end_char": 49
  },
  'nsubj',
  {
    "id": 1,
    "text": "Katy",
    "lemma": "Katy",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 9,
    "deprel": "nsubj",
    "misc": "",
    "start_char": 0,
    "end_char": 4
  }),
 ({
    "id": 1,
    "text": "Katy",
    "lemma": "Katy",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 9,
    "deprel": "nsubj",
    "misc": "",
    "start_char": 0,
    "end_char": 4
  },
  'flat',
  {
    "id": 2,
    "text": "B",
    "lemma": "B",
    "upos": "PROPN",
    "xpos": "NNP",
    "feats": "Number=Sing",
    "head": 1,
    "deprel": "flat",
    "misc": "",
    "start_char": 5,
    "end_char": 6
  }),
 ({
    "id": 1,
    "text": "Katy",
    "lem

In [51]:
np.random.choice([x for x in range(1, 176)], 3)

array([59, 80, 29])

In [53]:
all_albums = final_results.loc[:, "Album"].unique()

In [62]:
form = pd.read_hdf("C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album "
                   "Data\\new_unigram_data.h5", key="reduced_music_form")
form.loc[form["Word"] == "cocaine"]

,Artist,Album,Platform,Review id,Word id,Word,Desired
281091,ariel pink,pom pom,Pitchfork,112,1156,cocaine,-1
386312,marvin gaye,you're the man,NME,694,481,cocaine,-1
